### Input  --> Router  --> LLM decide chain ---> Chain ---> output

In [1]:
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms. 
You assume no prior knowledge. Here is the question\n{input}'''

In [2]:
expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a 
PhD level understanding of Physics. Here is the question\n{input}'''

## Route Prompts

In [16]:
prompt_infos = [
    {
        'name':'advanced physics',
        'description': 'Answers advanced physics questions',
        'prompt_template':expert_template
    },
    {
        'name':'beginner physics',
        'description': 'Answers basic beginner physics questions',
        'prompt_template':beginner_template
    },
]

In [17]:
from langchain.chains.llm import LLMChain
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama

In [26]:
chat = ChatOllama(model= "llama3.2:1b", temperature= 0.5)
destination_chains = {}  # key = "name" value = Chain object
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = LLMChain(llm = chat, prompt= prompt)
    destination_chains[name] = chain

In [19]:
default_prompt = ChatPromptTemplate.from_template(template= "{input}")
default_chain = LLMChain(llm = chat, prompt=default_prompt)

In [21]:
from langchain.chains.router.multi_prompt import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain_core.prompts.prompt import PromptTemplate

In [22]:
router_template = '''
Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
empty: Replies to empty questions
advanced physics: Answers advanced physics questions
beginner physics: Answers basic beginner physics questions

<< INPUT >>
{input}

<< OUTPUT >>
'''

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
C:\Users\spurusho\AppData\Local\Temp\ipykernel_23152\2711233452.py:1: SyntaxWarning: invalid escape sequence '\ '
  router_template = '''


In [23]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [24]:

router_chain = LLMRouterChain.from_llm(chat, router_prompt)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [27]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [28]:
chain.run("How do magnets work?")

C:\Users\spurusho\AppData\Local\Temp\ipykernel_23152\2065248458.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("How do magnets work?")




> Entering new MultiPromptChain chain...
advanced physics: {'input': 'What are the magnetic properties of a refrigerator magnet?'}
> Finished chain.


"A fascinating topic, my inquisitive students. As we delve into the realm of magnetic properties, let's explore the characteristics of a refrigerator magnet.\n\nRefrigerator magnets, also known as neodymium (NdFeB) magnets, are a type of permanent magnet that exhibits unique magnetic properties due to its composition and structure.\n\nThe key component in a refrigerator magnet is the neodymium iron boron (NdFeB) alloy. This alloy consists of:\n\n* Neodymium (Nd): A rare earth element with a high magnetic moment, responsible for the magnet's strength.\n* Iron (Fe): An essential element that provides ferromagnetic properties, allowing the magnet to retain its magnetic field over time.\n* Boron (B): A light element that contributes to the alloy's stability and resistance to corrosion.\n\nThe NdFeB alloy has a specific crystal structure, with a tetragonal lattice arrangement. This structure leads to:\n\n1. **High magnetic permeability**: The NdFeB alloy exhibits high magnetic permeability,

In [ ]:
# from operator import itemgetter
# from typing import Literal
# from typing_extensions import TypedDict

# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnableLambda, RunnablePassthrough
# from langchain_ollama.chat_models import ChatOllama
# llm = ChatOllama(model = "llama3.2:1b", template = 0.5)

# prompt_1 = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are an expert on animals."),
#         ("human", "{query}"),
#     ]
# )
# prompt_2 = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are an expert on vegetables."),
#         ("human", "{query}"),
#     ]
# )

# chain_1 = prompt_1 | llm | StrOutputParser()
# chain_2 = prompt_2 | llm | StrOutputParser()

# route_system = "Route the user's query to either the animal or vegetable expert."
# route_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", route_system),
#         ("human", "{query}"),
#     ]
# )


# class RouteQuery(TypedDict):
#     """Route query to destination."""
#     destination: Literal["animal", "vegetable"]


# route_chain = (
#     route_prompt
#     | llm.with_structured_output(RouteQuery)
#     | itemgetter("destination")
# )

# chain = {
#     "destination": route_chain,  # "animal" or "vegetable"
#     "query": lambda x: x["query"],  # pass through input query
# } | RunnableLambda(
#     # if animal, chain_1. otherwise, chain_2.
#     lambda x: chain_1 if x["destination"] == "animal" else chain_2,
# )

# chain.invoke({"query": "what color are carrots"})

'Carrots are typically orange in color. They can vary in shade from a deep, rich orange to a more pale or yellowish orange, but the classic and most well-known color is a vibrant orange.'